<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>توضیحات </div>
</font>
در این فایل قسمت هایی برای تست فاز اول پروژه قرار گرفته است. با اضافه شدن این قسمت ها به کد شما باید تمامی قسمت ها بدون اشکال اجرا شوند، در صورتی که شما فرضی به پروژه اضافه نکرده باشید و طبق استاندارد توضیح داده شده پیاده سازی کرده باشید، تمامی این قسمت ها بدون اشکال قابل اجرا هستند. اما در صورتی که قسمتی قابل اجرا نبود شما مجازید که با اعمال فرضیات خود کد تست را تغییر دهید تا تست ها به درستی اجرا شوند.
پس از اعمال تغییرات مورد نیاز در همین فایل
(بدون افزودن قسمت های قبلا پیاده سازی شده در پروژه)
فایل را در سایت کوئرا آپلود کنید.  
<strong>
توجه کنید که تغییرات شما باید جزئی و صرفا در جهت ایجاد همخوانی در روند تست با فرضیات شما باشند، پس به هیچ عنوان نباید روند کلی تست ها را تغییر دهید! همچنین تمامی تغییرات و دلیل انجام آن ها را در همین فایل در سلول های مناسب شرح دهید.
<br>
همچنین هرگونه تغییر در تعداد ورودی ها و چیدمان آن ها غیر مجاز میباشد.
</strong>
</font></div>

<div style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=2>
<h1 dir="rtl">قسمت اول و دوم<h1>
</div>

In [1]:
from src.models.TextPreparer import TextPreparer
from src.utils.construct_positional_indexes import construct_positional_indexes
from src.enums import *
from src.utils.load_index import load_index

In [2]:
text_preparer = TextPreparer()
manager = construct_positional_indexes()

100%|██████████| 1572/1572 [00:18<00:00, 84.06it/s] 


In [3]:
# This code was previously defined in MIRProjectPhase1.ipynb; uploaded with the rest of my codes for the first phase


def prepare_text(raw_text):
    prepared_text = text_preparer.prepare_text(raw_text)
    return prepared_text


def get_posting_list(word):
    posting_list = manager.corpus_index.get_posting_list(word)
    results = dict()
    for posting_list_item in posting_list:
        result = dict()
        result["text"] = posting_list_item.get_positions(Fields.TEXT)
        result["title"] = posting_list_item.get_positions(Fields.TITLE)
        results[posting_list_item.doc_id] = result
    return results


def get_words_with_bigram(bigram):
    return manager.bigram_index.get_words_with_bigram(bigram)


def add_document_to_indexes(docs_path, doc_num):
    manager.add_document_to_indexes(docs_path, doc_num)


def delete_document_from_indexes(docs_path, doc_num):
    manager.delete_document_from_indexes(docs_path, doc_num)


def save_index(destination):
    manager.save_index(destination)


def search(query, method="ltn-lnn", weight=2):
    return manager.search(
        query,
        method=Methods(method),
        field_weights={Fields.TEXT: 1.0, Fields.TITLE: weight},
        max_retrieved=15,
        correct_word=True,
    )


def detailed_search(title_query, text_query, method="ltn-lnn"):
    return manager.search(
        {Fields.TEXT: text_query, Fields.TITLE: title_query},
        method=Methods(method),
    )

In [4]:
word1 = 'فکری'
doc_id = 3014


word2 = 'هیلاندراس'
doc_id2 = 6752
bigram = 'لا'

def get_count (l):
    i = [1 for _,t in l.items() for q in t['text']]
    j = [1 for _,t in l.items() if 'title' in t.keys() for q in t['title']]
    return len (i) + len(j)


def test_prepare_text():
    print ("\n============ testing 'prepare_text' =============================================")
    raw_text = "کتابهای مناسبی نوشته شوند ! در راستای ارتقای . سطح آموزش کشور ؟ تلاش‌های زیادی صورت می‌گیرد"
    
    prepared_text = text_preparer.prepare_text(raw_text)    

    print("prepared text is :", prepared_text , "with length:" , len (prepared_text))
    
test_prepare_text()

def test_get_posting_list():
    
    print ("\n============ testing 'get_posting_list' =========================================")
    
    prepared_text = prepare_text(word1)[0]
    posting_list = get_posting_list(prepared_text)
    # posting_list = {3014:{'title':[...] , 'text':[...]}}
    
    
#     print ("posting list for input" , prepared_text, "is :", posting_list , "with length:" , len (posting_list))
    print ("number of ocurrences of the word", word1 , " in documents = ", get_count (posting_list))
    print ('docs with the word:' , sorted (list (posting_list.keys())))
    
test_get_posting_list()


def test_bigram():
    print ("\n============ testing 'get_words_with_bigram' ====================================")
    
    words_with_bigram = get_words_with_bigram(bigram)
    print ("returned list length:" , len (words_with_bigram))

    print ("checking word" , word2 , ":", word2 in words_with_bigram)
    

check_bigram = True
if check_bigram:
    test_bigram()

def test_doc_remove():
    
    print ("\n============ testing 'doc_remove' ================================================")
    
    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "before removing doc" , doc_id, ":" , len (posting_list))
    
    delete_document_from_indexes('data/Persian.xml', doc_id)
    
    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "after removing doc" , doc_id, ":" , len (posting_list))
    
    posting_list = get_posting_list(prepare_text(word2)[0])
    print ("length of posting list for word" , word2 , "before removing doc" , doc_id2, ":" , len (posting_list))
    
    delete_document_from_indexes('data/Persian.xml', doc_id2)
    
    posting_list = get_posting_list(prepare_text(word2)[0])
    print ("length of posting list for word" , word2 , "after removing doc" , doc_id2, ":" , len (posting_list))
    
test_doc_remove()

def test_doc_remove_bigram():
    print ("\n============ testing correct bigram removal ========================================")
    words_with_bigram = get_words_with_bigram(bigram)
    print ("returned list length:" , len (words_with_bigram))
    print ("checking word" , word2 , ":", word2 in words_with_bigram)
    

if check_bigram:
    test_doc_remove_bigram()

def test_doc_add():
    print ("\n============ testing 'doc_add' ================================================")
    
    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "before adding doc" , doc_id, ":" , len (posting_list))
    print ("number of ocurrences for ", word1, ":", get_count (posting_list))
    
    add_document_to_indexes('data/Persian.xml', doc_id)
    
    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "after adding doc" , doc_id, ":" , len (posting_list))
    print ("number of ocurrences for ", word1, ":", get_count (posting_list))
    
    
    posting_list = get_posting_list(prepare_text(word2)[0])
    print ("length of posting list for word" , word2 , "before adding doc" , doc_id2, ":" , len (posting_list))
    
    add_document_to_indexes('data/Persian.xml', doc_id2)
    
    posting_list = get_posting_list(prepare_text(word2)[0])
    print ("length of posting list for word" , word2 , "after adding doc" , doc_id2, ":" , len (posting_list))
    
    
test_doc_add()


def test_doc_add_bigram():
    
    words_with_bigram = get_words_with_bigram(bigram)
    print ("returned list length:" , len (words_with_bigram))
    print ("checking word" , word2 , ":", word2 in words_with_bigram)

if check_bigram:
    test_doc_remove_bigram()
    
def test_save_and_load ():
    print ("\n============ testing save and load methods ========================================")
    
    destination = "storage/backup/manager.pickle""

    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "before saving:" , len (posting_list))
    print ("number of ocurrences for ", word1, ":", get_count (posting_list))
    
    save_index(destination)
    manager = load_index(destination)
    
    posting_list = get_posting_list(prepare_text(word1)[0])
    print ("length of posting list for word" , word1 , "after loading:" , len (posting_list))
    print ("number of ocurrences for ", word1, ":", get_count (posting_list))
    
test_save_and_load ()


============ testing 'prepare_text' =============================================
prepared text is : ['کتاب', 'مناسب', 'نوشته_شوند', 'در', 'راستا', 'ارتقا', 'سطح', 'آموز', 'کشور', 'تلاش', 'زیاد', 'صور', 'می\u200cگیرد'] with length: 13

============ testing 'get_posting_list' =========================================
number of ocurrences of the word فکری  in documents =  168
docs with the word: [3014, 3099, 3103, 3119, 3197, 3217, 3229, 3359, 3373, 3404, 3429, 3654, 3699, 3776, 3777, 3798, 3826, 3879, 3897, 3938, 4002, 4062, 4248, 4275, 4321, 4335, 4382, 4388, 4391, 4398, 4400, 4460, 4636, 4650, 4671, 4718, 4726, 4743, 4805, 4843, 4864, 5192, 5308, 5381, 5486, 5554, 5571, 5707, 5720, 5820, 5967, 6014, 6052, 6088, 6229, 6294, 6417, 6418, 6475, 6522, 6568, 6572, 6609, 6629, 6634, 6710, 6735, 6749, 6752, 6753, 6791, 6847, 6848, 6907, 6931, 6944, 6959, 6973, 7133]

============ testing 'get_words_with_bigram' ====================================
returned list length: 2403
checking word هیل

<div style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=2>
<h1 dir="rtl">قسمت سوم<h1>
</div>

<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify">
تابع زیر چون در فاز پیش مشخص نشده بود  تازه نوشته شده. در واقع این تابع در داخل کد سرچ وجود دارد.
</div>

In [5]:
def correct_query(query: str):
    query_tokens = text_preparer.prepare_text(query)
    corrected_tokens = []
    for word in query_tokens:
        corrected_tokens.append(manager.correct_word(word))
    return corrected_tokens

In [6]:
def test_correct_query():

    ##################################
    ## Do not change this part
    ##################################
    query = "ابذار های فظایی و پیشرفته ناصا"
    ##################################

    result = correct_query(query)
    print(result)


test_correct_query()

['اذار', 'فا', 'و', 'پیشرفته', 'نا']


In [7]:
def test_search():

    ##################################
    ## Do not change this part
    ##################################
    query = 'سیاره های بزرگ "منظومه شمسی"'
    method = "ltc-lnc"
    ##################################

    relevant_docs = search(query, method)
    print(relevant_docs)


test_search()

[5724, 3903, 4255, 3411, 3416, 3414, 6629, 3415, 6266, 4259, 4376, 4627, 4659, 4573, 4624]


In [8]:
def test_detailed_search():

    ##################################
    ## Do not change this part
    ##################################
    title_query = "فهرست شهرهای ایران"
    text_query = "استان گیلان شهرستان لنگرود"
    ##################################

    relevant_docs = detailed_search(title_query, text_query)
    print(relevant_docs)


test_detailed_search()

[3760, 5264, 5236, 3758, 6369, 5644, 3874, 6394, 4275, 4094, 5381, 6159, 6417, 4339, 5932]


<div style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=2>
<h1 dir="rtl">قسمت چهارم<h1>
</div>

<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
در این قسمت به دلیل استفاده از utf8
در فایل های داده، در بعضی از کد های ارسال شده پردازش فایل ها با خطا مواجه میشود برای حل این مشکل بدون دست بردن در کد های ارسال شده میتوانیم از سلول زیر استفاده کنیم.  
درصورتی که شما نیازی به اعمال این تغییر ندارید میتوانید این سلول را در تست خود اجرا نکنید.
<strong>
در صورتی که نیاز به ایجاد تغییر در این سلول دارید حتما تغییرات خود را در یک سلول متنی شرح دهید!!!
</strong>
</div>

In [9]:
try:
    python_open
    print("Already done!")
except NameError:
    python_open = open

    def open(
        file,
        mode="r",
        buffering=-1,
        encoding=None,
        errors=None,
        newline=None,
        closefd=True,
        opener=None,
    ):
        encoding = "utf-8"
        return python_open(
            file,
            mode=mode,
            buffering=buffering,
            encoding=encoding,
            errors=errors,
            newline=newline,
            closefd=closefd,
            opener=opener,
        )

In [10]:
##################################
## Do not change this part
##################################
test_docs = ["all", 1, 2, 3]
# functions = {'R_Precision':R_Precision, 'F_measure':F_measure, 'MAP':MAP, 'NDCG': NDCG}
##################################

# for doc in test_docs:
#     print("{}\ndoc:\t{}".format('-'*30, doc))
#     for f in functions.keys():
#         out = functions[f](doc)
#         print("{:11}:\t{:.2f}".format(f, out))

<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
من قبلا این تابع را پیاده سازی کردم که دیگه همینجا استفاده می‌کنم
</div>

In [11]:
from src.metrics import evaluate_search_engine

In [12]:
for doc in test_docs:
    print("{}\ndoc:\t{}".format("-" * 30, doc))
    evaluate_search_engine(manager, method=Methods.LTN_LNN)

------------------------------
doc:	all
r_precision: 0.6105804602529432
ndcg: 0.6787020126128037
f_measure: 0.6279770638289564
map: 0.7569273771354816
------------------------------
doc:	1
r_precision: 0.6105804602529432
ndcg: 0.6787020126128037
f_measure: 0.6279770638289564
map: 0.7569273771354816
------------------------------
doc:	2
r_precision: 0.6105804602529432
ndcg: 0.6787020126128037
f_measure: 0.6279770638289564
map: 0.7569273771354816
------------------------------
doc:	3
r_precision: 0.6105804602529432
ndcg: 0.6787020126128037
f_measure: 0.6279770638289564
map: 0.7569273771354816


<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
نمونه خروجی:
</div>

------------------------------
    doc:	all
    R_Precision:	0.73
    F_measure  :	0.65
    MAP        :	0.63
    NDCG       :	0.75
    
------------------------------
    doc:	1
    R_Precision:	1.00
    F_measure  :	0.97
    MAP        :	0.94
    NDCG       :	0.96
    
------------------------------
    doc:	2
    R_Precision:	1.00
    F_measure  :	0.91
    MAP        :	0.83
    NDCG       :	0.90
    
------------------------------
    doc:	3
    R_Precision:	0.73
    F_measure  :	0.49
    MAP        :	0.29
    NDCG       :	0.48

<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
در این قسمت توابع امتیاز دهی شما مستقل از سایر قسمت های پروژه تست میشوند. برای این کار توابع جستجو مجددا با ساختار ساده ای پیاده سازی شده اند تا خروجی ثابتی را تولید کنند.
</div>

In [13]:
import numpy as np

from src.metrics import f_measure, ndcg_at_k, r_precision, average_precision
from src.utils.read_queries import read_queries

In [14]:
def calc_retrieved_relevance_and_num_relevant_docs(query_id):
    queries, relevants = read_queries(query_id)
    retrieved_docs = search("")
    retrieved_relevance = []
    relevants = relevants[0]
    for retrieved_doc in retrieved_docs:
        if retrieved_doc in relevants:
            retrieved_relevance.append(1)
        else:
            retrieved_relevance.append(0)
    num_relevant_docs = len(relevants)
    return retrieved_relevance, num_relevant_docs

In [15]:
def R_Precision(query_id):
    retrieved_relevance, num_relevant_docs = calc_retrieved_relevance_and_num_relevant_docs(
        query_id
    )
    return r_precision(retrieved_relevance, num_relevant_docs)


def F_measure(query_id):
    retrieved_relevance, num_relevant_docs = calc_retrieved_relevance_and_num_relevant_docs(
        query_id
    )
    return f_measure(retrieved_relevance, num_relevant_docs)


def MAP(query_id):
    retrieved_relevance, num_relevant_docs = calc_retrieved_relevance_and_num_relevant_docs(
        query_id
    )
    return average_precision(retrieved_relevance)


def NDCG(query_id):
    retrieved_relevance, num_relevant_docs = calc_retrieved_relevance_and_num_relevant_docs(
        query_id
    )
    return ndcg_at_k(
        retrieved_relevance,
        np.ones(len(retrieved_relevance)),
        num_relevant_docs,
    )

In [16]:
##################################
## Do not change this part
##################################
test_docs = [1, 2, 3]
rels = [
    [
        6753,
        7134,
        6978,
        7136,
        4530,
        6798,
        6885,
        5381,
        6900,
        4537,
        5509,
        6794,
        4094,
        6417,
        3666,
        5967,
    ],
    [6753, 5509, 4718, 6798, 6850, 6417, 6978, 6871],
    list(range(20)),
]
outputs = [
    {
        "R_Precision": 1.0,
        "F_measure": 0.967_741_935_483_871,
        "MAP": 0.9375,
        "NDCG": 0.963_564_011_026_350_9,
    },
    {
        "R_Precision": 0.444_444_444_444_444_4,
        "F_measure": 0.615_384_615_384_615_3,
        "MAP": 0.444_444_444_444_444_4,
        "NDCG": 0.631_380_202_279_965_8,
    },
    {"R_Precision": 0.0, "F_measure": 0.0, "MAP": 0.0, "NDCG": 0.0},
]

functions = {
    "R_Precision": R_Precision,
    "F_measure": F_measure,
    "MAP": MAP,
    "NDCG": NDCG,
}
##################################
idx = 0

ds = detailed_search
s = search


def detailed_search(title_query, text_query, method="ltn-lnn"):
    return rels[idx]


def search(query, method="ltn-lnn", weight=2):
    return rels[idx]


for f in functions.keys():
    print("{}\n{}:".format("-" * 30, f))
    idx = 0
    for doc in test_docs:
        out = functions[f](doc)
        expected = outputs[idx][f]
        print("{}:\t{:.2f}\t{}".format(doc, out, abs(out - expected) < 1e-3))
        idx += 1

detailed_search = ds
search = s

------------------------------
R_Precision:
1:	1.00	True
2:	0.44	True
3:	0.00	True
------------------------------
F_measure:
1:	1.00	False
2:	0.62	True
3:	0.00	True
------------------------------
MAP:
1:	1.00	False
2:	1.00	False
3:	0.00	True
------------------------------
NDCG:
1:	1.00	False
2:	1.00	False
3:	0.00	True


<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
فکر کنم شاید در تعریف تابع های ارزیابی یکم تفاوت وجود داره اما من تا جایی که می‌دونم توابع را درست پیاده سازی کردم.
</div>

<div dir="rtl" style="direction:rtl;line-height:300%;text-align:justify;" align="justify"><font face="XB Zar" size=5>
نمونه خروجی:
</div>

------------------------------
    R_Precision: 
    1:	1.00	True 
    2:	0.44	True 
    3:	0.00	True 
    
------------------------------
    F_measure: 
    1:	0.97	True
    2:	0.62	True
    3:	0.00	True

------------------------------
    MAP: 
    1:	0.94	True
    2:	0.44	True
    3:	0.00	True

------------------------------
    NDCG:
    1:	0.96	True
    2:	0.63	True
    3:	0.00	True
